In [6]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time
from scipy import stats
import numpy as np

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [8]:
def standardize_sample_wise(dataset):
    standardized = []
    for s in dataset:
        mean, std = s.mean(), s.std()
        s = (s - mean) / std
        standardized.append(s)
    return np.array(standardized)

x_train = standardize_sample_wise(x_train)
x_test = standardize_sample_wise(x_test)

In [9]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'selu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['selu']

In [10]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': Activation(keras.activations.elu),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=0.7) 
        }

        model_name = 'std_sample_wise_' + act + '_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=0,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation selu with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 15s 256us/step - loss: 0.4107 - acc: 0.8756 - val_loss: 0.1814 - val_acc: 0.9458
Epoch 2/100
60000/60000 [==============================] - 5s 88us/step - loss: 0.2216 - acc: 0.9338 - val_loss: 0.1192 - val_acc: 0.9645
Epoch 3/100
60000/60000 [==============================] - 6s 94us/step - loss: 0.1735 - acc: 0.9473 - val_loss: 0.1186 - val_acc: 0.9658
Epoch 4/100
60000/60000 [==============================] - 6s 97us/step - loss: 0.1536 - acc: 0.9530 - val_loss: 0.1152 - val_acc: 0.9643
Epoch 5/100
60000/60000 [==============================] - 5s 79us/step - loss: 0.1384 - acc: 0.9585 - val_loss: 0.0954 - val_acc: 0.9724
Epoch 6/100
60000/60000 [==============================] - 5s 84us/step - loss: 0.1264 - acc: 0.9623 - val_loss: 0.0900 - val_acc: 0.9745
Epoch 7/100
60000/60000 [==============================] - 5s 81us/s

60000/60000 [==============================] - 4s 62us/step - loss: 0.0600 - acc: 0.9820 - val_loss: 0.1113 - val_acc: 0.9778
Epoch 60/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.0607 - acc: 0.9829 - val_loss: 0.1071 - val_acc: 0.9796
Epoch 61/100
60000/60000 [==============================] - 4s 72us/step - loss: 0.0593 - acc: 0.9824 - val_loss: 0.1046 - val_acc: 0.9781
Epoch 62/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.0583 - acc: 0.9829 - val_loss: 0.1116 - val_acc: 0.9775
Epoch 63/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.0577 - acc: 0.9838 - val_loss: 0.1094 - val_acc: 0.9792
Epoch 64/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.0581 - acc: 0.9829 - val_loss: 0.1184 - val_acc: 0.9775
Epoch 65/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.0546 - acc: 0.9832 - val_loss: 0.1156 - val_acc: 0.9779
Epoch 66/100
60000/60000 [==============

Epoch 17/100
60000/60000 [==============================] - 4s 72us/step - loss: 0.0844 - acc: 0.9739 - val_loss: 0.0846 - val_acc: 0.9764
Epoch 18/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.0849 - acc: 0.9749 - val_loss: 0.0834 - val_acc: 0.9777
Epoch 19/100
60000/60000 [==============================] - 4s 69us/step - loss: 0.0805 - acc: 0.9751 - val_loss: 0.0869 - val_acc: 0.9782
Epoch 20/100
60000/60000 [==============================] - 4s 70us/step - loss: 0.0800 - acc: 0.9761 - val_loss: 0.0878 - val_acc: 0.9780
Epoch 21/100
60000/60000 [==============================] - 4s 71us/step - loss: 0.0816 - acc: 0.9753 - val_loss: 0.0792 - val_acc: 0.9782
Epoch 22/100
60000/60000 [==============================] - 4s 69us/step - loss: 0.0768 - acc: 0.9766 - val_loss: 0.0781 - val_acc: 0.9786
Epoch 23/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.0778 - acc: 0.9763 - val_loss: 0.0885 - val_acc: 0.9776
Epoch 24/100
60000/60000 [=

Epoch 76/100
60000/60000 [==============================] - 4s 74us/step - loss: 0.0590 - acc: 0.9829 - val_loss: 0.1053 - val_acc: 0.9793
Epoch 77/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0593 - acc: 0.9831 - val_loss: 0.1247 - val_acc: 0.9761
Epoch 78/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.0594 - acc: 0.9830 - val_loss: 0.1072 - val_acc: 0.9787
Epoch 79/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0542 - acc: 0.9841 - val_loss: 0.1133 - val_acc: 0.9801
Epoch 80/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.0557 - acc: 0.9837 - val_loss: 0.1058 - val_acc: 0.9792
Epoch 81/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.0569 - acc: 0.9836 - val_loss: 0.1218 - val_acc: 0.9775
Epoch 82/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.0562 - acc: 0.9842 - val_loss: 0.1175 - val_acc: 0.9783
Epoch 83/100
60000/60000 [=

Epoch 34/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0686 - acc: 0.9791 - val_loss: 0.0892 - val_acc: 0.9797
Epoch 35/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.0652 - acc: 0.9802 - val_loss: 0.0921 - val_acc: 0.9785
Epoch 36/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0664 - acc: 0.9798 - val_loss: 0.0847 - val_acc: 0.9781
Epoch 37/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.0646 - acc: 0.9811 - val_loss: 0.0940 - val_acc: 0.9799
Epoch 38/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0667 - acc: 0.9800 - val_loss: 0.0889 - val_acc: 0.9785
Epoch 39/100
60000/60000 [==============================] - 4s 70us/step - loss: 0.0656 - acc: 0.9803 - val_loss: 0.0915 - val_acc: 0.9791
Epoch 40/100
60000/60000 [==============================] - 4s 69us/step - loss: 0.0675 - acc: 0.9803 - val_loss: 0.0911 - val_acc: 0.9795
Epoch 41/100
60000/60000 [=

Epoch 93/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.0550 - acc: 0.9839 - val_loss: 0.1259 - val_acc: 0.9791
Epoch 94/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.0522 - acc: 0.9847 - val_loss: 0.1212 - val_acc: 0.9781
Epoch 95/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.0566 - acc: 0.9837 - val_loss: 0.1179 - val_acc: 0.9783
Epoch 96/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.0556 - acc: 0.9844 - val_loss: 0.1257 - val_acc: 0.9793
Epoch 97/100
60000/60000 [==============================] - 4s 73us/step - loss: 0.0526 - acc: 0.9853 - val_loss: 0.1067 - val_acc: 0.9792
Epoch 98/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.0579 - acc: 0.9842 - val_loss: 0.1212 - val_acc: 0.9778
Epoch 99/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.0563 - acc: 0.9841 - val_loss: 0.1142 - val_acc: 0.9792
Epoch 100/100
60000/60000 [

Epoch 51/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0642 - acc: 0.9815 - val_loss: 0.1106 - val_acc: 0.9753
Epoch 52/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.0594 - acc: 0.9820 - val_loss: 0.1081 - val_acc: 0.9790
Epoch 53/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0605 - acc: 0.9820 - val_loss: 0.1056 - val_acc: 0.9786
Epoch 54/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0612 - acc: 0.9812 - val_loss: 0.1000 - val_acc: 0.9789
Epoch 55/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0598 - acc: 0.9823 - val_loss: 0.0989 - val_acc: 0.9790
Epoch 56/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.0615 - acc: 0.9819 - val_loss: 0.1017 - val_acc: 0.9798
Epoch 57/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0611 - acc: 0.9822 - val_loss: 0.1060 - val_acc: 0.9790
Epoch 58/100
60000/60000 [=

Epoch 9/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1059 - acc: 0.9686 - val_loss: 0.0889 - val_acc: 0.9731
Epoch 10/100
60000/60000 [==============================] - 4s 64us/step - loss: 0.1047 - acc: 0.9684 - val_loss: 0.0863 - val_acc: 0.9754
Epoch 11/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0998 - acc: 0.9695 - val_loss: 0.0929 - val_acc: 0.9736
Epoch 12/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0962 - acc: 0.9710 - val_loss: 0.0853 - val_acc: 0.9758
Epoch 13/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.0919 - acc: 0.9727 - val_loss: 0.0798 - val_acc: 0.9757
Epoch 14/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0892 - acc: 0.9733 - val_loss: 0.0813 - val_acc: 0.9780
Epoch 15/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0888 - acc: 0.9730 - val_loss: 0.0891 - val_acc: 0.9774
Epoch 16/100
60000/60000 [==

60000/60000 [==============================] - 4s 61us/step - loss: 0.0574 - acc: 0.9835 - val_loss: 0.1156 - val_acc: 0.9801
Epoch 68/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0600 - acc: 0.9824 - val_loss: 0.1199 - val_acc: 0.9770
Epoch 69/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.0576 - acc: 0.9835 - val_loss: 0.1104 - val_acc: 0.9803
Epoch 70/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.0561 - acc: 0.9836 - val_loss: 0.1135 - val_acc: 0.9796
Epoch 71/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.0576 - acc: 0.9835 - val_loss: 0.1163 - val_acc: 0.9769
Epoch 72/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.0619 - acc: 0.9822 - val_loss: 0.1182 - val_acc: 0.9791
Epoch 73/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.0583 - acc: 0.9834 - val_loss: 0.1101 - val_acc: 0.9799
Epoch 74/100
60000/60000 [==============